In [15]:
# THIS FUNCTION READS THE DATA FILE CONTAINING THE INFORMATION RE EACH PLAYER

import pandas as pd

# read data
data = (pd.read_csv("clean-data.csv")
        .reset_index(drop=True))

num_players = len(data.index)

print("num possible players is %s" % (num_players))


num possible players is 523


In [ ]:
import numpy as np

# HELPFUL DATA 
# these can be used for calculating points and costs and are also used in the constraint_checking function
points = data['Points'] 
cost = data['Cost']
    

# create lists with all elements initialised to 0
gk = np.zeros(num_players)
mid = np.zeros(num_players)
defe = np.zeros(num_players)
stri = np.zeros(num_players)

for i in range(num_players):
    if data['Position'][i] == 'GK':
        gk[i] = 1
    elif data['Position'][i] == 'DEF':
        defe[i] = 1
    elif data['Position'][i] == 'MID':
        mid[i] = 1
    elif data['Position'][i] == 'STR':
        stri[i]=1
  

In [ ]:
def check_constraints(individual):
     
    broken_constraints = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints+=1
        print("total players is %s " %(c1))
        
    
    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual)) 
    if c2 > 100:
        broken_constraints+=1
        print("cost is %s " %(c2))
    
    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints+=1
        print("goalies is %s " %(c3))
    
    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints+=1
        print("DEFE is %s " %(c4))
            
    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3: 
        broken_constraints+=1
        print("MID is %s " %(c5))
        
    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1: 
        broken_constraints+=1
        print("STR is %s " %(c6))
        
    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]
    
    totalpoints = np.sum(np.multiply(points, individual))
        
        
    print("total broken constraints: %s" %(broken_constraints))
    print("total points: %s" %(totalpoints))
    print("total cost is %s" %(c2))
    print("selected players are %s" %(selectedPlayers))
    
    return broken_constraints, totalpoints

In [ ]:

import array
import random
import numpy
import matplotlib
import matplotlib.pyplot as plt


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import pandas as pd

In [ ]:

MAX_COST = 100
MAX_PLAYERS = 11

MUTPB = 0.1
CXPB = 0.7
POPSIZE = 200
NGEN=300
TNSIZE=2



TOTAL_COST =0
for i in range(num_players):
    TOTAL_COST += cost[i]

TOTAL_POINTS =0
for i in range(num_players):
    TOTAL_POINTS += points[i]



print(num_players, TOTAL_POINTS, TOTAL_COST)

In [ ]:
def myInitialisationFunction(icls, size):

    # first create an individual with all bits set to 0
    ind = icls(numpy.zeros(size))
    

    current_cost=0
    current_players=0
    current_gk=0
    current_defe=0
    current_mid=0
    current_stri=0
    
    
    team_gk = 1
    team_defe = random.randint(4,5)
    team_mid = random.randint(3,4)
    team_stri = 11 - (team_defe + team_mid + team_gk) #make sure the remainging spaces get filled up

    
    defe_range = list(range(0, 180))
    mid_range = list(range(181, 376))
    stri_range = list(range(377, 466))
    gk_range = list(range(467, 523))
    
    random.shuffle(defe_range)
    random.shuffle(mid_range)
    random.shuffle(stri_range)
    random.shuffle(gk_range)
    
    x = 0
    while team_defe > 0:
        player = defe_range[x]
        new_cost = cost[player]
        
        if current_cost + new_cost <= MAX_COST:
            ind[player] = 1
            current_cost += new_cost
            x += 1
            team_defe -= 1
            current_players += 1
            current_defe += 1
            
    x = 0
    while team_mid > 0:
        player = mid_range[x]
        new_cost = cost[player]
        
        if current_cost + new_cost <= MAX_COST:
            ind[player] = 1
            current_cost += new_cost
            x += 1
            team_mid -= 1
            current_players += 1
            current_mid += 1
            
    x = 0
    while team_stri > 0:
        player = stri_range[x]
        new_cost = cost[player]
        
        if current_cost + new_cost <= MAX_COST:
            ind[player] = 1
            current_cost += new_cost
            x += 1
            team_stri -= 1
            current_players += 1
            current_stri += 1
            
    x = 0
    while team_gk > 0:
        player = gk_range[x]
        new_cost = cost[player]
        
        if current_cost + new_cost <= MAX_COST:
            ind[player] = 1
            current_cost += new_cost
            x += 1
            team_gk -= 1
            current_players += 1
            current_gk += 1
        

    #print("team cost is %s, players is %s, gk=%s, def=%s, mid=%s, str=%s" %(current_cost, current_players, current_gk, current_defe, current_mid, current_stri))
    return ind

In [ ]:
## SET UP THE EA
    
# define the fitness class and creare an individual class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


# create a toolbox
toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)

# USE THIS LINE IF YOU WANT TO USE THE CUSTOM INIT FUNCTION
toolbox.register("individual", myInitialisationFunction, creator.Individual, num_players)

#  a population consist of a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [ ]:
# DEFINE FITNESS FOR KNAPSACK
# fitness function definition - death penalty

def evalTeam(individual):
    
    current_cost = 0
    current_point = 0.0
    current_players = 0
    current_gk = 0
    current_def = 0
    current_mid = 0
    current_str = 0
    
    for player in range(num_players):
        if (individual[player]==1):
            current_cost += cost[player]
            current_point += points[player]
            current_players += 1
            
            if gk[player]==1:
                current_gk += 1  
            elif defe[player]==1:
                current_def += 1      
            elif mid[player]==1:
                current_mid += 1       
            elif stri[player]==1:
                current_str += 1
                
    if  current_cost > MAX_COST or current_players != MAX_PLAYERS or current_gk != 1 or current_def not in range(1, 5) or current_mid not in range(1,5) or current_str not in range(1,3):
        return 0,
    
    return  current_point,

In [ ]:
# register all operators we need with the toolbox
toolbox.register("evaluate", evalTeam)

#toolbox.register("mate", tools.cxUniform, indpb=0.1)
toolbox.register("mate", tools.cxOnePoint)
#toolbox.register("mate", tools.cxTwoPoint)

#toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)

toolbox.register("select", tools.selTournament, tournsize=TNSIZE)
#toolbox.register("select", tools.selRoulette)

In [ ]:
# create an dataframe that has 3 columns to record the important data from each run
column_names = ['popsize', 'fitness', 'genMaxFound']
df = pd.DataFrame(columns = column_names)

In [ ]:
# main function

def main():
    
    # choose a population size: e.g. 200
    pop = toolbox.population(n=POPSIZE)
    
    # keep track of the single best solution found
    hof = tools.HallOfFame(1)
 
    # create a statistics object: we can log what ever statistics we want using this. We use the numpy Python library
    # to calculate the stats and label them with convenient labels
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    # run the algorithm: we need to tell it what parameters to use
    # cxpb = crossover probability; mutpb = mutation probability; ngen = number of iterations
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof


# ##############################
# # run the main function 
# pop, log, hof = main()

# ##############################


# best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list


# # look in the logbook to see what generation this was found at

# max = log.select("max")  # max fitness per generation stored in log

# for i in range(NGEN):  # set to ngen
#         fit = max[i]
#         if fit == best:
#             break      
            
# df = df.append({'popsize': POPSIZE , 'fitness': best, 'genMaxFound':i}, ignore_index=True)
        
# print("max fitness found is %s at generation %s" % (best, i))

In [ ]:
df

In [ ]:

# code for plotting

gen = log.select("gen")
fit_max = log.select("max")
fit_min = log.select("min")
fit_avg = log.select("avg")

fig, ax1 = plt.subplots()
line1 = ax1.plot(gen, fit_max, "b-", label="max Fitness", color="r")
line2 = ax1.plot(gen, fit_min, "b-", label="min Fitness", color="b")
line3 = ax1.plot(gen , fit_avg, "b-", label="avg Fitness", color="g")
ax1.set_xlabel("Generations")
ax1.set_ylabel("Fitness", color="b")
for tl in ax1.get_yticklabels():
    tl.set_color("b")
ax1.set_ylim(12000000,14000000)
    
lns = line1+line2+line3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc="center right")


In [ ]:
individual = []

for i in hof:
    for p in i:
        if p == 0.0:
            individual.append(0)
        elif p == 1:
            individual.append(1)



check_constraints(individual)

# Experiment:  test POPSIZE from 50-150 in steps of 10

In [ ]:
# Experiment:  test POPSIZE from 50-150 in steps of 10


# create an dataframe that has 3 columns to record the important data from each run
column_names = ['popsize', 'fitness', 'genMaxFound']
df = pd.DataFrame(columns = column_names)


for  POPSIZE in range(500, 1501, 500):
        # repeat EA 10x for each parameter
        
        print('POPSIZE = ', POPSIZE)
        
        for reps in range(5):
            print('REP = ', reps)
            pop,log,hof = main()
            # extract the best fitness
            best = hof[0].fitness.values[0]
            # save the generation this fitness was first found
            max = log.select('max')
            for gen in range(NGEN):  
                if max[gen] == best:
                    break   
            
         
            df = df.append({'popsize': POPSIZE , 'fitness': best, 'genMaxFound':gen}, ignore_index=True)

In [ ]:
df

In [ ]:
# code for printing statistics and plots
print(df.groupby('popsize').mean())
print(df.groupby('popsize').median())
print(df.groupby('popsize').std())

# plot the boxplot of fitness per population size
boxplot = df.boxplot(column=['fitness'], by=['popsize'])

# plot genMaxFound per population size
boxplot = df.boxplot(column=['genMaxFound'], by=['popsize'])

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import shapiro

# select two subsets of fitness data, one where popsize = 50 and the other where popsize = 150
p50 = df.fitness[df.popsize == 50.0]
p150= df.genMaxFound[df.popsize == 150.0]

# we need to convert these subsets to lists to pass to the statistics functions
p50l= p50.tolist()
p150l = p150.tolist()

# check for normality
stat,p = shapiro(p50l)
print('normality p=%f' %(p))

In [ ]:
#now apply the  appropriate statistical test depending on the result of the above
stat,p = ttest_ind(p50l,p150l)
#stat,p = mannwhitneyu(p50l,p150l)

print('p=%f' %(p))

# Experiment:  test MUTPB from 0 to 1 in steps of 0.2

In [ ]:
# create an dataframe that has 3 columns to record the important data from each run
column_names = ['mutpb', 'fitness', 'genMaxFound']
df = pd.DataFrame(columns = column_names)


for  MUTPB in numpy.arange(0.0, 1.2, 0.2):
        # repeat EA 10x for each parameter
        print('MUTPB = ', MUTPB)
        
        for reps in range(5):
            print('REP = ', reps)
            pop,log,hof = main()
            # extract the best fitness
            best = hof[0].fitness.values[0]
            # save the generation this fitness was first found
            max = log.select('max')
            for gen in range(NGEN):  
                if max[gen] == best:
                    break   
            
         
            df = df.append({'mutpb': MUTPB , 'fitness': best, 'genMaxFound':gen}, ignore_index=True)

In [ ]:
df

In [ ]:
# code for printing statistics and plots
print(df.groupby('mutpb').mean())
print(df.groupby('mutpb').median())
print(df.groupby('mutpb').std())

# plot the boxplot of fitness per population size
boxplot = df.boxplot(column=['fitness'], by=['mutpb'])

# plot genMaxFound per population size
boxplot = df.boxplot(column=['genMaxFound'], by=['mutpb'])

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import shapiro

# select two subsets of fitness data, one where popsize = 50 and the other where popsize = 150
m04 = df.fitness[df.mutpb == 0.4]
m08 = df.genMaxFound[df.mutpb == 0.8]

# we need to convert these subsets to lists to pass to the statistics functions
m04l= m04.tolist()
m08l = m08.tolist()

# check for normality
stat,p = shapiro(m04l)
print('normality p=%f' %(p))

In [ ]:
#now apply the  appropriate statistical test depending on the result of the above
#stat,p = ttest_ind(m04l,m08l)
stat,p = mannwhitneyu(m04l,m08l)

print('p=%f' %(p))

# Experiment:  Test Crossover cxOnePoint/cxTwoPoint/cxUniform

In [ ]:
column_names = ['crossover', 'fitness', 'genMaxFound']
df3 = pd.DataFrame(columns = column_names)

In [ ]:
for reps in range(5):
    pop,log,hof = main()
    # extract the best fitness
    best = hof[0].fitness.values[0]
    # save the generation this fitness was first found
    max = log.select('max')
    for gen in range(NGEN):  
        if max[gen] == best:
            break   
            
         
    df3 = df3.append({'crossover': 'cxTwoPoint' , 'fitness': best, 'genMaxFound':gen}, ignore_index=True)

In [ ]:
df3

In [ ]:
# code for printing statistics and plots
print(df.groupby('crossover').mean())
print(df.groupby('crossover').median())
print(df.groupby('crossover').std())

# plot the boxplot of fitness per population size
boxplot = df.boxplot(column=['fitness'], by=['crossover'])

# plot genMaxFound per population size
boxplot = df.boxplot(column=['genMaxFound'], by=['crossover'])

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import shapiro

# select two subsets of fitness data, one where popsize = 50 and the other where popsize = 150
cOne = df.fitness[df.crossover == 'cxOnePoint']
cUni = df.genMaxFound[df.crossover == 'cxUniform']

# we need to convert these subsets to lists to pass to the statistics functions
cOnel= cOne.tolist()
cUnil = cUni.tolist()

# check for normality
stat,p = shapiro(cOnel)
print('normality p=%f' %(p))

In [ ]:
#now apply the  appropriate statistical test depending on the result of the above
stat,p = ttest_ind(cOnel,cUnil)
#stat,p = mannwhitneyu(p50l,p150l)

print('p=%f' %(p))

# Experiment:  Test Tournament Size 2/4/6/8

In [ ]:
column_names = ['tournSize', 'fitness', 'genMaxFound']
df4 = pd.DataFrame(columns = column_names)

In [ ]:
for TNSIZE in range(2, 11, 4):
    print('TNSIZE', TNSIZE)
    for reps in range(5):
        print('reps', reps)
        pop,log,hof = main()
        # extract the best fitness
        best = hof[0].fitness.values[0]
        # save the generation this fitness was first found
        max = log.select('max')
        for gen in range(NGEN):  
            if max[gen] == best:
                break   
            
         
    df4 = df4.append({'tournSize': TNSIZE , 'fitness': best, 'genMaxFound':gen}, ignore_index=True)

In [ ]:
df4

In [ ]:
# code for printing statistics and plots
print(df4.groupby('tournSize').mean())
print(df4.groupby('tournSize').median())
print(df4.groupby('tournSize').std())

# plot the boxplot of fitness per population size
boxplot = df4.boxplot(column=['fitness'], by=['tournSize'])

# plot genMaxFound per population size
boxplot = df4.boxplot(column=['genMaxFound'], by=['tournSize'])

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import shapiro

# select two subsets of fitness data, one where popsize = 50 and the other where popsize = 150
cOne = df4.fitness[df.crossover == 'cxOnePoint']
cUni = df4.genMaxFound[df.crossover == 'cxUniform']

# we need to convert these subsets to lists to pass to the statistics functions
cOnel= cOne.tolist()
cUnil = cUni.tolist()

# check for normality
stat,p = shapiro(cOnel)
print('normality p=%f' %(p))

In [ ]:
#now apply the  appropriate statistical test depending on the result of the above
stat,p = ttest_ind(cOnel,cUnil)
#stat,p = mannwhitneyu(p50l,p150l)

print('p=%f' %(p))

# Experiment:  Test Selection Operator

In [ ]:
column_names = ['selOp', 'fitness', 'genMaxFound']
df5 = pd.DataFrame(columns = column_names)

In [ ]:
for reps in range(5):
    print('reps', reps)
    pop,log,hof = main()
    # extract the best fitness
    best = hof[0].fitness.values[0]
    # save the generation this fitness was first found
    max = log.select('max')
    for gen in range(NGEN):  
        if max[gen] == best:
            break   
            
         
    df5 = df5.append({'selOp': 'Roullete' , 'fitness': best, 'genMaxFound':gen}, ignore_index=True)

In [ ]:
df5

In [ ]:
# code for printing statistics and plots
print(df5.groupby('selOp').mean())
print(df5.groupby('selOp').median())
print(df5.groupby('selOp').std())

# plot the boxplot of fitness per population size
boxplot = df5.boxplot(column=['fitness'], by=['selOp'])

# plot genMaxFound per population size
boxplot = df5.boxplot(column=['genMaxFound'], by=['selOp'])

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import shapiro

# select two subsets of fitness data, one where popsize = 50 and the other where popsize = 150
tour = df5.fitness[df5.selOp == 'Tournament']
roul = df5.genMaxFound[df5.selOp == 'Roullete']

# we need to convert these subsets to lists to pass to the statistics functions
tourL= tour.tolist()
roulL = roul.tolist()

# check for normality
stat,p = shapiro(tourL)
print('normality p=%f%%' %(p*100))

In [ ]:
#now apply the  appropriate statistical test depending on the result of the above
#stat,p = ttest_ind(tourL,roulL)
stat,p = mannwhitneyu(tourL,roulL)

print('p=%f%%' %(p*100))

# Code to read in saved solution

**You only need this code if you have written your coursework using another language and need to check the saved solution. Otherwise, ignore this section**

The code below expects you have saved you solution to a csv file. The file should contain 523 rows, each of which has a single value  set to 0 or 1.


Once you have read in your data, you will need to:
- read in the data file with the player information
- run the code in the cell above to create the lists cost/points/gk/defe/mid/stri
- run the check_constraints function passing the "individual" read from your file using the code below

In [ ]:
import csv

with open('pathtofile/mysolution.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    individual = []
    for row in readCSV:
        value=row[0]
        individual.append(int(value))

print(individual)

# check length
num_players = len(individual.index)

if num_players != 523:
    print("the solution file does not contain the correct number of variables")
    
